### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/687_Trocas8_June2021_PEAKS_146/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/687_Trocas8_June2021_PEAKS_146


In [4]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [5]:
ls 

 687_Trocas8_June2021_PEAKS_146_peptide.csv     'de novo only peptides.csv'
 687_Trocas8_June2021_PEAKS_146_proteins.csv     peptides_1_1_0.mzid
 687_Trocas8_June2021_PEAKS_146_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [6]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup687 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/687_Trocas8_June2021_PEAKS_146/687_Trocas8_June2021_PEAKS_146_peptide.csv")

# remove redundant rows
peaksdb687 = pd.DataFrame.drop_duplicates(peaksdbdup687)

print(peaksdb687.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb687.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb687['#Spec']
del peaksdb687['Accession']
del peaksdb687['PTM']
del peaksdb687['AScore']

mean_length = peaksdb687['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup687))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb687))

#look at the dataframe
peaksdb687.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 13', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 13', 'Accession', 'PTM', 'AScore'],
      dtype='object')
9.27536231884058
# redundant peaksdb peptides in combined dataframe 69
# nonredundant peaksdb peptides in combined dataframe 69


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,VLEGNEQFINAAK,48.94,1431.7357,13,-0.2,716.8750,68.94,135000.0,13,12132,20210624_Trocas8_687_SMCP66_50%_DDA_120min.raw
1,LSSPATLNSR,48.81,1044.5564,10,0.2,523.2856,45.86,14800000.0,13,8198,20210624_Trocas8_687_SMCP66_50%_DDA_120min.raw
2,LSSPATLN(+.98)SR,44.89,1045.5404,10,0.8,523.7779,48.87,154000.0,13,8735,20210624_Trocas8_687_SMCP66_50%_DDA_120min.raw
3,LATVISPR,38.00,855.5178,8,-0.5,428.7660,60.40,9020000.0,13,10750,20210624_Trocas8_687_SMCP66_50%_DDA_120min.raw
4,LATVLSPR,38.00,855.5178,8,-0.5,428.7660,60.40,9020000.0,13,10750,20210624_Trocas8_687_SMCP66_50%_DDA_120min.raw


In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb687['A'] = peaksdb687['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb687['C'] = peaksdb687['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb687['D'] = peaksdb687['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb687['E'] = peaksdb687['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb687['F'] = peaksdb687['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb687['G'] = peaksdb687['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb687['H'] = peaksdb687['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb687 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb687['I'] = peaksdb687['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb687['K'] = peaksdb687['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb687['L'] = peaksdb687['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb687['M'] = peaksdb687['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb687['N'] = peaksdb687['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb687['P'] = peaksdb687['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb687['Q'] = peaksdb687['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb687['R'] = peaksdb687['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb687['S'] = peaksdb687['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb687['T'] = peaksdb687['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb687['V'] = peaksdb687['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb687['W'] = peaksdb687['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb687['Y'] = peaksdb687['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb687['c-carb'] = peaksdb687['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb687['m-oxid'] = peaksdb687['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb687['n-deam'] = peaksdb687['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb687['q-deam'] = peaksdb687['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb687['stripped_peptide'] = peaksdb687['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb687['stripped_length'] = peaksdb687['stripped_peptide'].apply(len)

peaksdb687['NAAF_num.'] = peaksdb687['Area'] / peaksdb687['stripped_length']

# total the number of modifications in sequence
peaksdb687['ptm-total'] = peaksdb687['c-carb'] + peaksdb687['m-oxid'] + peaksdb687['n-deam'] + peaksdb687['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb687['stripped_IL']= peaksdb687['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb687.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB.csv")

# check out the results
peaksdb687.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,VLEGNEQFINAAK,48.94,1431.7357,13,-0.2,716.8750,68.94,135000.0,13,12132,...,0,0,0,0,0,VLEGNEQFINAAK,13,1.038462e+04,0,VLEGNEQFLNAAK
1,LSSPATLNSR,48.81,1044.5564,10,0.2,523.2856,45.86,14800000.0,13,8198,...,0,0,0,0,0,LSSPATLNSR,10,1.480000e+06,0,LSSPATLNSR
2,LSSPATLN(+.98)SR,44.89,1045.5404,10,0.8,523.7779,48.87,154000.0,13,8735,...,0,0,0,1,0,LSSPATLNSR,10,1.540000e+04,1,LSSPATLNSR
3,LATVISPR,38.00,855.5178,8,-0.5,428.7660,60.40,9020000.0,13,10750,...,0,0,0,0,0,LATVISPR,8,1.127500e+06,0,LATVLSPR
4,LATVLSPR,38.00,855.5178,8,-0.5,428.7660,60.40,9020000.0,13,10750,...,0,0,0,0,0,LATVLSPR,8,1.127500e+06,0,LATVLSPR


In [8]:
# keep only stripped peptide I/L and NAAF
db_687 = peaksdb687[['stripped_IL', 'Area', 'NAAF_num.']]

db_687.set_index('stripped_IL')

# write modified dataframe to new txt file
db_687.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/687_SMCP_50_GF75_PDB_ILnaafs.csv")

db_687.head()

,stripped_IL,Area,NAAF_num.
0,VLEGNEQFLNAAK,135000.0,1.038462e+04
1,LSSPATLNSR,14800000.0,1.480000e+06
2,LSSPATLNSR,154000.0,1.540000e+04
3,LATVLSPR,9020000.0,1.127500e+06
4,LATVLSPR,9020000.0,1.127500e+06


### Exporting txt files of stripped peptides at confidence cutoffs:

In [9]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep687moddup = peaksdb687[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep687dup = peaksdb687[["stripped_peptide"]]

# deduplicate both of these lists
pep687mod = pep687moddup.drop_duplicates()
pep687 = pep687dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 687:", len(pep687moddup))
print("Deduplicated modified PeaksDB peptides in 687:", len(pep687mod))
print("Total stripped PeaksDB peptides in 687:", len(pep687dup))
print("Deduplicated stripped PeaksDB peptides in 687:", len(pep687))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep687.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep687.head()

Total modified PeaksDB peptides in 687: 69
Deduplicated modified PeaksDB peptides in 687: 69
Total stripped PeaksDB peptides in 687: 69
Deduplicated stripped PeaksDB peptides in 687: 68


,stripped_peptide
0,VLEGNEQFINAAK
1,LSSPATLNSR
3,LATVISPR
4,LATVLSPR
5,SCL


## NAAF correction and exporting files with AA and PTM totals:

In [10]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb687['A'].sum(),
        'C': peaksdb687['C'].sum(),
        'D': peaksdb687['D'].sum(),
        'E': peaksdb687['E'].sum(),
        'F': peaksdb687['F'].sum(),
        'G': peaksdb687['G'].sum(),
        'H': peaksdb687['H'].sum(),
        'I': peaksdb687['I'].sum(),
        'K': peaksdb687['K'].sum(),
        'L': peaksdb687['L'].sum(),
        'M': peaksdb687['M'].sum(),
        'N': peaksdb687['N'].sum(),
        'P': peaksdb687['P'].sum(),
        'Q': peaksdb687['Q'].sum(),
        'R': peaksdb687['R'].sum(),
        'S': peaksdb687['S'].sum(),
        'T': peaksdb687['T'].sum(),
        'V': peaksdb687['V'].sum(),
        'W': peaksdb687['W'].sum(),
        'Y': peaksdb687['Y'].sum(),
        'c-carb': peaksdb687['c-carb'].sum(),
        'm-oxid': peaksdb687['m-oxid'].sum(),
        'n-deam': peaksdb687['n-deam'].sum(),
        'q-deam': peaksdb687['q-deam'].sum(),
        'Total area': peaksdb687['Area'].sum(),
        'Total length': peaksdb687['stripped_length'].sum()
       }

totalpeaksdb687 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb687['% C w/ carb'] = totalpeaksdb687['c-carb'] / totalpeaksdb687['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb687['% M w/ oxid'] = totalpeaksdb687['m-oxid'] / totalpeaksdb687['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb687['% N w/ deam'] = totalpeaksdb687['n-deam'] / totalpeaksdb687['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb687['% Q w/ deam'] = totalpeaksdb687['q-deam'] / totalpeaksdb687['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb687['NAAF denom.'] = totalpeaksdb687['Total area'] / totalpeaksdb687['Total length']

# write modified dataframe to new txt file
totalpeaksdb687.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_totals.csv")

totalpeaksdb687.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,51,3,17,26,21,41,17,41,60,76,...,4,3,2,100232867.0,633,1.0,0.333333,0.12,0.133333,158345.761453


In [11]:
# use the calculated NAAF factor (in totalpeaksdb687 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 158345.761453

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb687['NAAF factor'] = (peaksdb687['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb687_NAAF = peaksdb687[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor SMCP the AAs to normalize its abundance SMCP peak area and peptide length

peaksdb687_NAAF['A-NAAF20'] = peaksdb687_NAAF['A'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['C-NAAF20'] = peaksdb687_NAAF['C'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['D-NAAF20'] = peaksdb687_NAAF['D'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['E-NAAF20'] = peaksdb687_NAAF['E'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['F-NAAF20'] = peaksdb687_NAAF['F'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['G-NAAF20'] = peaksdb687_NAAF['G'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['H-NAAF20'] = peaksdb687_NAAF['H'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['I-NAAF20'] = peaksdb687_NAAF['I'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['K-NAAF20'] = peaksdb687_NAAF['K'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['L-NAAF20'] = peaksdb687_NAAF['L'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['M-NAAF20'] = peaksdb687_NAAF['M'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['N-NAAF20'] = peaksdb687_NAAF['N'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['P-NAAF20'] = peaksdb687_NAAF['P'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['Q-NAAF20'] = peaksdb687_NAAF['Q'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['R-NAAF20'] = peaksdb687_NAAF['R'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['S-NAAF20'] = peaksdb687_NAAF['S'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['T-NAAF20'] = peaksdb687_NAAF['T'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['V-NAAF20'] = peaksdb687_NAAF['V'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['W-NAAF20'] = peaksdb687_NAAF['W'] * peaksdb687['NAAF factor']
peaksdb687_NAAF['Y-NAAF20'] = peaksdb687_NAAF['Y'] * peaksdb687['NAAF factor']

# multiply the NAAF20 factor SMCP the PTMs normalize its abundance SMCP peak area and peptide length

peaksdb687_NAAF['ccarb-NAAF20'] = peaksdb687_NAAF['c-carb'] * peaksdb687_NAAF['NAAF factor']
peaksdb687_NAAF['moxid-NAAF20'] = peaksdb687_NAAF['m-oxid'] * peaksdb687_NAAF['NAAF factor']
peaksdb687_NAAF['ndeam-NAAF20'] = peaksdb687_NAAF['n-deam'] * peaksdb687_NAAF['NAAF factor']
peaksdb687_NAAF['qdeam-NAAF20'] = peaksdb687_NAAF['q-deam'] * peaksdb687_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb687_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_naaf.csv")

peaksdb687_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,VLEGNEQFINAAK,0.065582,2,0,0,2,1,1,0,1,...,0.000000,0.000000,0.000000,0.065582,0.0,0.0,0.0,0.0,0.000000,0.0
1,LSSPATLNSR,9.346635,1,0,0,0,0,0,0,0,...,9.346635,28.039904,9.346635,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,LSSPATLNSR,0.097256,1,0,0,0,0,0,0,0,...,0.097256,0.291767,0.097256,0.000000,0.0,0.0,0.0,0.0,0.097256,0.0
3,LATVISPR,7.120494,1,0,0,0,0,0,0,1,...,7.120494,7.120494,7.120494,7.120494,0.0,0.0,0.0,0.0,0.000000,0.0
4,LATVLSPR,7.120494,1,0,0,0,0,0,0,0,...,7.120494,7.120494,7.120494,7.120494,0.0,0.0,0.0,0.0,0.000000,0.0


In [12]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb687 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb687_NAAF['NAAF factor'].sum(),
        'A': peaksdb687_NAAF['A-NAAF20'].sum(),
        'C': peaksdb687_NAAF['C-NAAF20'].sum(),
        'D': peaksdb687_NAAF['D-NAAF20'].sum(),
        'E': peaksdb687_NAAF['E-NAAF20'].sum(),
        'F': peaksdb687_NAAF['F-NAAF20'].sum(),
        'G': peaksdb687_NAAF['G-NAAF20'].sum(),
        'H': peaksdb687_NAAF['H-NAAF20'].sum(),
        'I': peaksdb687_NAAF['I-NAAF20'].sum(),
        'K': peaksdb687_NAAF['K-NAAF20'].sum(),
        'L': peaksdb687_NAAF['L-NAAF20'].sum(),
        'M': peaksdb687_NAAF['M-NAAF20'].sum(),
        'N': peaksdb687_NAAF['N-NAAF20'].sum(),
        'P': peaksdb687_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb687_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb687_NAAF['R-NAAF20'].sum(),
        'S': peaksdb687_NAAF['S-NAAF20'].sum(),
        'T': peaksdb687_NAAF['T-NAAF20'].sum(),
        'V': peaksdb687_NAAF['V-NAAF20'].sum(),
        'W': peaksdb687_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb687_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb687_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb687_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb687_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb687_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb687_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb687_NAAF['% C w/ carb.'] = totalpeaksdb687_NAAF['c-carb'] / totalpeaksdb687_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb687_NAAF['% M w/ oxid'] = totalpeaksdb687_NAAF['m-oxid'] / totalpeaksdb687_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb687_NAAF['% N w/ deam'] = totalpeaksdb687_NAAF['n-deam'] / totalpeaksdb687_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb687_NAAF['% Q w/ deam'] = totalpeaksdb687_NAAF['q-deam'] / totalpeaksdb687_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb687_NAAF['NAAF check'] = totalpeaksdb687_NAAF['NAAF'] / 158345.761453

# write modified dataframe to new txt file, same name + totals
totalpeaksdb687_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/679_SMCP_50_GF75_PDB/687_SMCP_50_GF75_PDB_naaf_totals.csv")

totalpeaksdb687_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,77.207151,38.859775,0.434029,1.347412,1.062711,0.288018,1.363185,0.611157,31.762172,1.186585,...,0.69726,0.434029,0.050985,0.097713,0.0,1.0,0.057225,0.009356,0.0,0.000488
